In [202]:
import numpy as np
import time
import pandas as pd

IN_COLAB = False
import sys
if IN_COLAB:
  
  scripts_dir = '/drive/My Drive/Colab Notebooks/scripts/'
  sys.path.insert(1, scripts_dir)
# from opfunu.cec_basic.cec2014_nobias import *
# from mealpy.swarm_based.PSO import BasePSO

# insert at 1, 0 is the script path (or '' in REPL)
else:
    sys.path.insert(1, 'scripts')

from geneticalgorithm import geneticalgorithm as ga
from geneticalgorithm1 import geneticalgorithm1 as ga1
from geneticalgorithmOptd import geneticalgorithmOptd as gaOptd

from PSO import BasePSO
from BBO import BaseBBO

import pandas as pd

# import benchmark_func as bf

import importlib
bf = importlib.import_module("benchmark_func")

In [203]:
aa = locals()
if not 'Npar' in locals():
    Npar = 0.3
else:
    Npar = aa['Npar']

dimension = 10
population_size = 100
max_iter = 200
num_runs = 5
seeds = np.random.randint(0, 1000, num_runs)
varbound=np.array([[-100, 100]]*dimension)

In [204]:
timestr = time.strftime("%Y%m%d-%H%M%S")
filename = f'results/high_pop/Run_Results-{timestr}'
pivot_filename = "resultFile_d{}_pop{}_iter{}_runs{}_Npar{}--{}.xlsx"\
    .format(dimension, population_size, max_iter, num_runs, Npar, timestr)

one_args = getattr(bf, "__oneArgument__")
two_args = getattr(bf, "__twoArgument__")

In [205]:
result = {}
result["function"] = None
result["run"] = None
result["seed"] = None
result["algo"] = None
result["pop_size"] = None
result["g_opt_val"] = None
result["func_val"] = None

result["max_iter"] = None
result["best_iter"] = None

results = []


In [206]:
def createPop(dimension=2, population_size=15):
  var=np.zeros(dimension) 

  pop=np.array([np.zeros(dimension)]*population_size)
  for p in range(0, population_size):
      for i in range(0, dimension):
          var[i]=np.random.randint(varbound[i][0],\
                  varbound[i][1]+1) 
      pop[p] = var
  data = {}
  data['pop'] = pop.copy()

  # print("$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
  # print(data['pop'])

  return data

In [207]:
def normGa(obj_func, pop_data=None, seed = 777, terminal_value=None):
  algorithm_param = {'max_num_iteration': max_iter,\
                   'population_size':population_size,\
                   'mutation_probability':0.2,\
                   'elit_ratio': 0.1,\
                   'crossover_probability': .7,\
                   'parents_portion': 0.3,\
                   'crossover_type':'uniform',\
                   'max_iteration_without_improv':None,\
                   'terminal_value': terminal_value}
  model=ga(function=obj_func,dimension=dimension,variable_type='real',variable_boundaries=varbound, random_seed=seed, algorithm_parameters=algorithm_param)
  model.run('GA_data.dat', data=pop_data)

  data = {}
  data['best_sol'] = model.best_variable
  data['best_fit'] = model.best_function
  data['best_iter'] = model.terminal_value_iteration
  # data['max_iter'] = model.iterate
  return data

In [208]:
def imprvGa(obj_func, pop_data=None, seed = 777, terminal_value=None):
  algorithm_param = {'max_num_iteration': max_iter,\
                   'population_size':population_size,\
                   'mutation_probability':0.3,\
                   'elit_ratio': 0.1,\
                   'crossover_probability': .8,\
                   'parents_portion': Npar,\
                   'crossover_type':'uniform',\
                   'max_iteration_without_improv':None,\
                   'terminal_value': terminal_value}
  model=gaOptd(function=obj_func,dimension=dimension,variable_type='real',variable_boundaries=varbound, random_seed=seed, algorithm_parameters=algorithm_param)
  model.run('GA_data.dat', data=pop_data)

  data = {}
  data['best_sol'] = model.best_variable
  data['best_fit'] = model.best_function
  data['best_iter'] = model.terminal_value_iteration
  # data['max_iter'] = model.iterate
  # data['best_sol'] = best_pos1
  # data['best_sol'] = best_pos1
  return data

In [209]:
def pso(obj_func, pop_data=None, seed = 777, terminal_value=None):
  lb = varbound[:, 0].tolist()
  ub = varbound[:, 1].tolist()

  verbose = False
  
  model = BasePSO(obj_func, lb, ub, verbose, max_iter, population_size, random_seed=seed, terminal_value=terminal_value)  # Remember the keyword "problem_size"
  best_sol, best_fit, list_loss1 = model.train('pso_data.dat', data=pop_data)
  data = {}
  data['best_sol'] = best_sol
  data['best_fit'] = best_fit
  data['best_iter'] = model.terminal_value_iteration
  # data['max_iter'] = max_iter
  # data['best_sol'] = best_pos1
  # data['best_sol'] = best_pos1
  return data

In [210]:
def bbo(obj_func, pop_data=None, seed = 777, terminal_value=None):
  lb = varbound[:, 0].tolist()
  ub = varbound[:, 1].tolist()

  verbose = False

  model = BaseBBO(obj_func, lb, ub, verbose, max_iter, population_size, random_seed=seed, terminal_value=terminal_value)  # Remember the keyword "problem_size"
  best_sol, best_fit, list_loss1 = model.train('bbo_data.dat', data=pop_data)
  data = {}
  data['best_sol'] = best_sol
  data['best_fit'] = best_fit
  data['best_iter'] = model.terminal_value_iteration
  # data['max_iter'] = max_iter
  # data['best_sol'] = best_pos1
  # data['best_sol'] = best_pos1
  return data


In [211]:
for func in two_args:
  class_ = getattr(bf, func)
  instance = class_(dimension)
  count = 1
  for seed in seeds:
    np.random.seed(seed)
    obj_func = instance.get_func_val
    pop_data = createPop(dimension, population_size)

    print("NormGA")
    res = normGa(obj_func, pop_data)

    result = {}
    result["function"] = func
    result["run"] = count
    result["seed"] = seed
    result["algo"] = "NormGA"
    result["func_val"] = res["best_fit"]
    result["pop_size"] = population_size
    # result["max_iter"] = max_iter
    result["best_iter"] = res['best_iter']

    results.append(result)

    print("imprvGA")
    res = imprvGa(obj_func, pop_data)

    result = {}
    result["function"] = func
    result["run"] = count
    result["seed"] = seed
    result["algo"] = "imprvGA"
    result["func_val"] = res["best_fit"]
    result["pop_size"] = population_size
    # result["max_iter"] = max_iter
    result["best_iter"] = res['best_iter']

    results.append(result)

    print("PSO")
    res = pso(obj_func, pop_data)

    result = {}
    result["function"] = func
    result["run"] = count
    result["seed"] = seed
    result["algo"] = "PSO"
    result["func_val"] = res["best_fit"]
    result["pop_size"] = population_size
    # result["max_iter"] = max_iter
    result["best_iter"] = res['best_iter']

    results.append(result)

    print("BBO")
    res = bbo(obj_func, pop_data)

    result = {}
    result["function"] = func
    result["run"] = count
    result["seed"] = seed
    result["algo"] = "BBO"
    result["func_val"] = res["best_fit"]
    result["pop_size"] = population_size
    # result["max_iter"] = max_iter
    result["best_iter"] = res['best_iter']

    results.append(result)

    

    count += 1


NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
imprvGA
PSO
BBO
NormGA
impr

In [212]:
pd.DataFrame.from_dict(results, orient='columns').to_csv(f'{filename}.csv')

In [213]:
results_df = pd.DataFrame(results)


In [214]:
algos = results_df.algo.unique()
functions = results_df.function.unique()

In [215]:
describe_result = {}
describe_results = []
describe_results_df = None

count = 0
for i in range(len(algos)):
    for j in range(len(functions)):
        describe_result = {}
        new_df = results_df.loc[(results_df['function'] == functions[j]) & (results_df['algo'] == algos[i])]
        describe = new_df.describe().loc[['mean','std'], 'func_val'].tolist()
        describe_result['function'] = functions[j]
        describe_result['algo'] = algos[i]
        describe_result['mean'] = describe[0]
        describe_result['std'] = describe[1]
        describe_results.append(describe_result.copy())
    

In [216]:
func_global_opts = {}
for func in two_args:
    
    class_ = getattr(bf, func)
    instance = class_(dimension)
    func_global_opts[instance.func_name] = instance.global_optimum_solution

In [217]:
for data in describe_results:
    data['global_val'] = func_global_opts[data['function']]

In [218]:
describe_results_df = pd.DataFrame(describe_results)

In [219]:
describe_results_df.head(50)

,function,algo,mean,std,global_val
0,Ackley,NormGA,1.992920e+01,6.866089e-02,0.00000
1,Sphere,NormGA,1.547268e+03,5.313177e+02,0.00000
2,Rosenbrock,NormGA,2.869164e+07,1.459289e+07,0.00000
3,StyblinskiTang,NormGA,2.846140e+05,1.626589e+05,-391.66165
4,DeJongsF1,NormGA,1.547268e+03,5.313177e+02,0.00000
5,DeJongsF2,NormGA,2.869164e+07,1.459289e+07,0.00000
6,Ellipsoid,NormGA,1.230284e+07,1.196075e+07,0.00000
7,KTablet,NormGA,5.517181e+06,3.518927e+06,0.00000
8,WeightedSphere,NormGA,5.419350e+03,1.922628e+03,0.00000
9,HyperEllipsodic,NormGA,5.419350e+03,1.922628e+03,0.00000


In [220]:
pivot_table_df = pd.pivot_table(describe_results_df,

        index=['function', 'global_val'],

        columns=['algo'],

        values=['mean', 'std'],

        aggfunc=sum)
pivot_table_df

mean                              \
algo                                     BBO        NormGA           PSO   
function            global_val                                             
Ackley               0.00000    1.998575e+01  1.992920e+01  1.999178e+01   
DeJongsF1            0.00000    2.205298e+03  1.547268e+03  8.846199e+02   
DeJongsF2            0.00000    1.664378e+08  2.869164e+07  4.798475e+06   
Ellipsoid            0.00000    1.289366e+06  1.230284e+07  1.189240e+07   
Griewank             0.00000    5.728796e-01  3.185522e-01  2.034869e-01   
HyperEllipsodic      0.00000    1.541630e+04  5.419350e+03  5.291932e+03   
KTablet              0.00000    1.092200e+07  5.517181e+06  5.085614e+06   
Michalewicz         -1.80130   -6.083995e+00 -5.940219e+00 -2.742462e+00   
Rastrigin            0.00000    2.349131e+03  1.804686e+03  1.230354e+03   
Rosenbrock           0.00000    1.664378e+08  2.869164e+07  4.798475e+06   
Schwefel            -418.98290 -5.668474e+02 -5.330362e+02 -4.489965e+02   
Sphere               0.00000    2.205298e+03  1.547268e+03  8.846199e+02   
StyblinskiTang      -391.66165  1.465172e+06  2.846140e+05  5.406518e+04   
SumOfDifferentPower  0.00000    6.981440e+09  2.263388e+10  4.766131e+09   
WeightedSphere       0.00000    1.541630e+04  5.419350e+03  5.291932e+03   
XinSheYang           0.00000    4.008719e-02  7.818315e-02  6.159970e-01   
Zakharov             0.00000   -3.717747e+01 -4.732035e+01  1.302020e+01   

                                                       std                \
algo                                 imprvGA           BBO        NormGA   
function            global_val                                             
Ackley               0.00000    1.998434e+01  8.533967e-03  6.866089e-02   
DeJongsF1            0.00000    3.130326e+03  8.409508e+02  5.313177e+02   
DeJongsF2            0.00000    1.855391e+08  5.512889e+07  1.459289e+07   
Ellipsoid            0.00000    5.061826e+07  4.426175e+05  1.196075e+07   
Griewank             0.00000    1.060770e+00  1.930771e-01  1.137501e-01   
HyperEllipsodic      0.00000    1.750226e+04  3.283431e+03  1.922628e+03   
KTablet              0.00000    2.259112e+07  4.075551e+06  3.518927e+06   
Michalewicz         -1.80130   -3.296492e+00  8.805935e-01  6.815151e-01   
Rastrigin            0.00000    3.293414e+03  6.992045e+02  3.377664e+02   
Rosenbrock           0.00000    1.855391e+08  5.512889e+07  1.459289e+07   
Schwefel            -418.98290 -3.948083e+02  1.962062e+01  2.905888e+01   
Sphere               0.00000    3.130326e+03  8.409508e+02  5.313177e+02   
StyblinskiTang      -391.66165  2.759801e+06  5.597297e+05  1.626589e+05   
SumOfDifferentPower  0.00000    7.069985e+12  4.611396e+09  1.266085e+10   
WeightedSphere       0.00000    1.750226e+04  3.283431e+03  1.922628e+03   
XinSheYang           0.00000    2.428414e-01  4.706311e-03  5.353521e-02   
Zakharov             0.00000    7.260878e+03  5.540141e+01  4.835047e+01   

                                                            
algo                                     PSO       imprvGA  
function            global_val                              
Ackley               0.00000    5.348732e-03  1.716461e-02  
DeJongsF1            0.00000    4.896436e+02  1.095121e+03  
DeJongsF2            0.00000    3.866890e+06  1.266563e+08  
Ellipsoid            0.00000    8.841518e+06  8.280532e+06  
Griewank             0.00000    1.147253e-01  4.727941e-01  
HyperEllipsodic      0.00000    2.407052e+03  6.223564e+03  
KTablet              0.00000    2.135565e+06  7.737826e+06  
Michalewicz         -1.80130    2.786038e-01  7.312685e-01  
Rastrigin            0.00000    5.065069e+02  8.231975e+02  
Rosenbrock           0.00000    3.866890e+06  1.266563e+08  
Schwefel            -418.98290  1.899167e+01  5.666346e+01  
Sphere               0.00000    4.896436e+02  1.095121e+03  
StyblinskiTang      -391.66165  3.768082e+04  1.464339e+06  
SumOfDifferentPower  0

In [221]:
func_global_opts

{'Ackley': 0,
 'Sphere': 0,
 'Rosenbrock': 0,
 'StyblinskiTang': -391.66165,
 'DeJongsF1': 0,
 'DeJongsF2': 0,
 'Ellipsoid': 0.0,
 'KTablet': 0.0,
 'WeightedSphere': 0.0,
 'HyperEllipsodic': 0.0,
 'SumOfDifferentPower': 0.0,
 'Griewank': 0.0,
 'Michalewicz': -1.8013,
 'Rastrigin': 0.0,
 'Schwefel': -418.9829,
 'XinSheYang': 0.0,
 'Zakharov': 0.0}

In [222]:
pivot_table_df.keys()

MultiIndex([('mean',     'BBO'),
            ('mean',  'NormGA'),
            ('mean',     'PSO'),
            ('mean', 'imprvGA'),
            ( 'std',     'BBO'),
            ( 'std',  'NormGA'),
            ( 'std',     'PSO'),
            ( 'std', 'imprvGA')],
           names=[None, 'algo'])

In [223]:
pivot_table_df.to_excel(pivot_filename)

In [224]:
aa = "dsj{}-{}={}".format(2, 3, 4)

In [225]:
aa

'dsj2-3=4'

In [226]:
# describe_results_df.to_excel("globals.xlsx")